In [1]:
import pandas as pd
import re
import math
import openpyxl

In [2]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# AOPA

## Load output - from AOPAdata - get cleancali

### Data prep - clinic name ready in cleancali.xlsx

In [3]:
frs = pd.read_excel('output.xlsx', sheet_name = 'CALIFORNIA')

In [4]:
frs.head(2)

,Clinic_name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100","Arcadia , CA 91006",(626) 431-2890,Fax (626) 431-2892,mailto:ryan@crowncityortho.com,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics ❹,3227 Professional Dr,"Auburn , CA 95602",(530) 885-9922,mailto:sierrapo@gv.net,NaN,NaN,"BAKERSFIELD TUCSON Alicia Lenhart, RFOM",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# [x.lower() for x in dataframe['Provider Organization Name (Legal Business Name)']]

In [6]:
frs['Clinic_name'] = frs['Clinic_name'].str.replace(' C ', '').str.replace('❹', '').str.replace(' S ', '')

In [7]:
frs.to_excel('cleancali.xlsx',index=False)

##### Attempt for cleaning pipeline, havn't achieve so ...

In [8]:
# state = 'CA'

In [9]:
# frame = pd.read_excel('cleancali.xlsx')

In [10]:
# frame.columns

In [11]:
# frame.head(2)

In [12]:
# keepf = frame[(frame['Address'].str.contains(state)) | 
#           (frame['County'].str.contains(state)) | 
#           (frame['Unnamed: 3'].str.contains(state))]

In [13]:
# keepf

In [14]:
# def starts_with_number(s):
#     return re.match(r'^\d', s) is not None

In [15]:
# column_to_check = 'Address'
# new_data = []

# for index, row in keepf.iterrows():
#     value = row[column_to_check]
#     # print(value)
#     new_row = [None] + row.tolist() if not starts_with_number(value) else row.tolist()
#     new_data.append(new_row)

# new_df = pd.DataFrame(new_data, columns=[''] + df.columns.tolist())

##### ... manual cleaning for formatted column 

## Load cleancali2 - city, state, zip columns in cleancali3.xlsx

In [16]:
frame2 = pd.read_excel('cleancali2.xlsx')

In [17]:
frame2.head(2)

,Clinic_name,Address,City，State，Zipcode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100","Arcadia , CA 91006",(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,"Auburn , CA 95602",(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
frame2['City，State，Zipcode']

0                             Arcadia ,  CA 91006
1                             Auburn  ,  CA 95602
2                          Aliso Viejo, CA 92656 
3                         Arroyo Grande, CA 93420
4                      Bakersfield, CA 93301-1816
                          ...                    
170                     Ventura  ,  CA 93003-7710
171                            Vista  ,  CA 92081
172                       Vista  ,  CA 92081-6056
173    Whittier Medical Plz Whittier  ,  CA 90601
174                       Whittier, CA 90606-2583
Name: City，State，Zipcode, Length: 175, dtype: object

In [19]:
type(frame2['City，State，Zipcode'][37])

float

In [20]:
formatted_values = []

for value in frame2['City，State，Zipcode']:
    if isinstance(value, float) and math.isnan(value):
        city = ""
        state = ""
        zip_code = ""

    # Convert the value to string if it's not already
    if not isinstance(value, str):
        value = str(value)
        city = ""
        state = ""
        zip_code = ""
        
    if value.strip():
        parts = re.split(", |,\xa0", value)
        city = parts[0].strip()
        
        if len(parts) == 1:  # If only city name is present
            state = "CA"
            zip_code = ""
        else:
            state_zip = parts[1].split("A ")
            state = state_zip[0].strip() + 'A'
            zip_code = state_zip[1].strip()
            
        formatted_row = {
            'City': city,
            'State': state,
            'ZipCode': zip_code
        }
        
        formatted_values.append(formatted_row)

In [21]:
formatted_df = pd.DataFrame(formatted_values)

In [22]:
formatted_df.head(2)

,City,State,ZipCode
0,Arcadia,CA,91006
1,Auburn,CA,95602


In [23]:
temp_df3 = pd.concat([frame2, formatted_df], axis=1)

In [24]:
temp_df3.columns

Index(['Clinic_name', 'Address', 'City，State，Zipcode', 'Phone', 'FAX', 'Email',
       'Website', 'Name', 'Other_info', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'City', 'State', 'ZipCode'],
      dtype='object')

In [25]:
column_order = ['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'FAX', 'Email',
       'Website', 'Name', 'Other_info', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'City，State，Zipcode']
temp_df3 = temp_df3[column_order]

In [26]:
temp_df3 = temp_df3.drop(columns=['City，State，Zipcode'])

In [27]:
temp_df3['City'] = ["" if cell=='nan' else cell for cell in temp_df3['City']]

In [28]:
temp_df3['ZipCode'] = [cell[:5] for cell in temp_df3['ZipCode']]

In [29]:
temp_df3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# temp_df3.to_excel('cleancali3.xlsx',index=False)

## Update cleancali3 - clean rows/cols to combine multiple df

In [31]:
# frame3 = pd.read_excel('cleancali3.xlsx')
frame3 = temp_df3.copy()

In [32]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
frame3["ZipCode"] = frame3["ZipCode"].apply(lambda x: str(x) if not pd.isna(x) else "")

In [34]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
[x.lower().strip() for x in frame3["Clinic_name"]][:15]

['molina orthopedic labs inc./dba crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics &amp; orthotics',
 'achilles prosthetics &amp; orthotics inc.',
 'hanger clinic',
 'hanger clinic',
 'human designs prosthetic &amp; orthotic laboratory inc.',
 'north county prosthetics &amp; orthotics',
 'hanger clinic',
 'townsend design (thuasne usa)   +',
 'valley institute of prosthetics &amp; orthotics',
 'supply inc.',
 'hanger clinic',
 'prosthetic &amp; orthotic group los angeles']

In [36]:
def remove_amp(s):
    return s.replace("&amp;", "&").replace("+","")

In [37]:
def extract_y(s):
    match = re.search(r'/dba\s+(.+)', s)
    if match:
        return match.group(1)
    return s

In [38]:
frame3["Clinic_name"] = frame3["Clinic_name"].apply(lambda x: extract_y(x))
frame3["Clinic_name"] = frame3["Clinic_name"].apply(lambda x: remove_amp(x))

In [39]:
[x.lower().strip() for x in frame3["Clinic_name"]][:15]

['crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics & orthotics',
 'achilles prosthetics & orthotics inc.',
 'hanger clinic',
 'hanger clinic',
 'human designs prosthetic & orthotic laboratory inc.',
 'north county prosthetics & orthotics',
 'hanger clinic',
 'townsend design (thuasne usa)',
 'valley institute of prosthetics & orthotics',
 'supply inc.',
 'hanger clinic',
 'prosthetic & orthotic group los angeles']

In [40]:
cols_tolower = ['Clinic_name', 'Address', 'City', 'State', 'ZipCode']

In [41]:
frame3["Clinic_name"][:5].tolist()

['Crown City Orthopedic ',
 'Sierra Prosthetics -Orthotics   ',
 'Medequip Inc. ',
 'North County Prosthetics & Orthotics   ',
 'Achilles Prosthetics & Orthotics Inc.']

In [42]:
for col in cols_tolower:
    frame3[col] = frame3[col].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [43]:
frame3["Clinic_name"][:5].tolist()

['crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics & orthotics',
 'achilles prosthetics & orthotics inc.']

In [44]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,crown city orthopedic,"610 n santa anita ave, ste 100",arcadia,ca,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,sierra prosthetics -orthotics,3227 professional dr,auburn,ca,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


# NPI

In [45]:
dataframe = pd.read_excel('list_caliclinic.xlsx')

In [46]:
dataframe.head(2)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,City,State2,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number,Zip5,Address,taxonomy
0,1548468614,2,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400,94587,33374 DOWE AVE,335E00000X
1,1386652774,2,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3460 ROBIN LN,CAMERON PARK,CA,95682,5306764856,95682,3460 ROBIN LN,335E00000X


In [47]:
dataframe.columns

Index(['NPI', 'Entity Type Code',
       'Provider Organization Name (Legal Business Name)',
       'Provider First Line Business Practice Location Address', 'City',
       'State2', 'Provider Business Practice Location Address Postal Code',
       'Provider Business Practice Location Address Telephone Number', 'Zip5',
       'Address', 'taxonomy'],
      dtype='object')

In [48]:
column_order = ['Provider Organization Name (Legal Business Name)','Address','City',
                'State2','Zip5', 'Provider Business Practice Location Address Telephone Number',
                'NPI', 'taxonomy', 'Entity Type Code', 'Provider First Line Business Practice Location Address',
               'Provider Business Practice Location Address Postal Code']
dataframe = dataframe[column_order]

In [49]:
dataframe.rename(columns={'Provider Organization Name (Legal Business Name)': 'Clinic_name', 
                            'State2': 'State', 'Zip5': 'ZipCode',
                           'Provider Business Practice Location Address Telephone Number': 'Phone'}, inplace=True)

In [50]:
dataframe.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,NPI,taxonomy,Entity Type Code,Provider First Line Business Practice Location Address,Provider Business Practice Location Address Postal Code
0,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400,1548468614,335E00000X,2,33374 DOWE AVE,94587
1,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3460 ROBIN LN,CAMERON PARK,CA,95682,5306764856,1386652774,335E00000X,2,3460 ROBIN LN,95682


In [51]:
for col in cols_tolower:
    dataframe[col] = dataframe[col].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [52]:
dataframe.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,NPI,taxonomy,Entity Type Code,Provider First Line Business Practice Location Address,Provider Business Practice Location Address Postal Code
0,dimension prosthetics & orthotics,33374 dowe ave,union city,ca,94587,5103243400,1548468614,335E00000X,2,33374 DOWE AVE,94587
1,"hanger prosthetics & orthotics west, inc.",3460 robin ln,cameron park,ca,95682,5306764856,1386652774,335E00000X,2,3460 ROBIN LN,95682


In [53]:
frame3.head(3)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,crown city orthopedic,"610 n santa anita ave, ste 100",arcadia,ca,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,sierra prosthetics -orthotics,3227 professional dr,auburn,ca,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN
2,medequip inc.,27 brookline,aliso viejo,ca,92656,(949) 443-4414,NaN,soren@medequiportho.com,http://www.medequiportho.com/,"Soren Lind, President","Kasey Bancroft, O "" ce Manager","Brian Keim, Area Manager Trish Terena, Sales M...",NaN,NaN


# AOPA-NPI

In [54]:
dataframe['combineid'] = dataframe['Clinic_name'].str.split().str[0] + dataframe['ZipCode'].astype(str)
frame3['combineid'] = frame3['Clinic_name'].str.split().str[0] + frame3['ZipCode'].astype(str)

In [55]:
dataframe['combineid2'] = dataframe['Clinic_name'].str.split().str[0] + dataframe['Address'].str.split().str[0]
frame3['combineid2'] = frame3['Clinic_name'].str.split().str[0] + frame3['Address'].str.split().str[0]

In [56]:
dataframe['combineid3'] = dataframe['ZipCode'].astype(str) + dataframe['Address'].str.split().str[0]
frame3['combineid3'] = frame3['ZipCode'].astype(str) + frame3['Address'].str.split().str[0]

In [57]:
merged1 = pd.merge(dataframe, frame3, on=['combineid'], how='inner')
merged2 = pd.merge(dataframe, frame3, on=['combineid2'], how='inner')
merged3 = pd.merge(dataframe, frame3, on=['combineid3'], how='inner')

In [58]:
merged1 = merged1.fillna('NaN')
merged2 = merged2.fillna('NaN')
merged3 = merged3.fillna('NaN')

In [59]:
merged_df = pd.concat([merged1, merged2, merged3])

In [60]:
merged_df.shape

(292, 34)

In [61]:
merged_df.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 'combineid',
       'combineid2_x', 'combineid3_x', 'Clinic_name_y', 'Address_y', 'City_y',
       'State_y', 'ZipCode_y', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'combineid2_y', 'combineid3_y', 'combineid_x', 'combineid2',
       'combineid_y', 'combineid3'],
      dtype='object')

In [62]:
merged_df = merged_df[['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 
       'Clinic_name_y', 'Address_y', 'City_y',
       'State_y', 'ZipCode_y', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]

In [63]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(132, 25)

In [64]:
# merged_df.to_excel('temp.xlsx',index=False)

## Clean AOPA-NPI

In [65]:
# frame4 = pd.read_excel('temp.xlsx')
frame4 = merged_df.copy()

In [66]:
frame4.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code',
       'Clinic_name_y', 'Address_y', 'City_y', 'State_y', 'ZipCode_y',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [67]:
npi_list = dataframe['NPI'].tolist()
merged_list = frame4['NPI'].tolist()

In [68]:
left_values = [item for item in npi_list if item not in merged_list]

In [69]:
len(left_values)

269

In [70]:
unmerged_frame = dataframe[dataframe['NPI'].isin(left_values)]

In [71]:
frame4.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code',
       'Clinic_name_y', 'Address_y', 'City_y', 'State_y', 'ZipCode_y',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [72]:
frame4 = frame4[['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'NPI']]

In [73]:
frame4.rename(columns={'Clinic_name_x': 'Clinic_name', 'Address_x': 'Address', 'City_x': 'City',
                       'State_x': 'State', 'ZipCode_x': 'ZipCode', 'Phone_x': 'Phone'}, inplace=True)

In [74]:
unmerged_frame.columns

Index(['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'NPI',
       'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 'combineid',
       'combineid2', 'combineid3'],
      dtype='object')

In [75]:
unmerged_frame = unmerged_frame[['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'NPI']]

In [76]:
frame5 = pd.concat([frame4, unmerged_frame], ignore_index=True)

In [77]:
# frame5.to_excel('temp3.xlsx',index=False)

## Add Clinician info

In [78]:
clidf = pd.read_excel('comb_indorg.xlsx')

In [79]:
clidf.columns

Index(['NPI_x', 'Entity Type Code_x',
       'Provider Organization Name (Legal Business Name)_x',
       'Provider First Line Business Practice Location Address', 'City_x',
       'Provider Business Practice Location Address Postal Code_x',
       'Provider Business Practice Location Address Telephone Number_x',
       'Authorized Official Last Name_x', 'Authorized Official First Name_x',
       'Authorized Official Middle Name_x',
       'Authorized Official Telephone Number_x', 'Is Organization Subpart_x',
       'Zip5_x', 'Address_x', 'taxonomy_x', 'fml'],
      dtype='object')

In [80]:
clidf.head(2)

,NPI_x,Entity Type Code_x,Provider Organization Name (Legal Business Name)_x,Provider First Line Business Practice Location Address,City_x,Provider Business Practice Location Address Postal Code_x,Provider Business Practice Location Address Telephone Number_x,Authorized Official Last Name_x,Authorized Official First Name_x,Authorized Official Middle Name_x,Authorized Official Telephone Number_x,Is Organization Subpart_x,Zip5_x,Address_x,taxonomy_x,fml
0,1013275874,2,ELITE BIOMECHANICAL DESIGN,2208 5TH AVE,OROVILLE,959655816,5305346913,CASEY,MICHAEL,PATRICK,5308946913,N,95965,2208 5TH AVE,335E00000X,"NICHOLAS,D.,MADSEN"
1,1023188943,2,JENNINGS ORTHOPEDIC INC,10683 MAGNOLIA AVE,RIVERSIDE,925051893,9513522029,JENNINGS,HENRY,ALVIN,9513522029,N,92505,10683 MAGNOLIA AVE,224P00000X,"HENRY,ALVIN,JENNINGS"


In [81]:
clidf.rename(columns={'Provider Organization Name (Legal Business Name)_x': 'Clinic_name', 
                            'Address_x': 'Address', 'City_x': 'City',
                           'Zip5_x': 'ZipCode'}, inplace=True)

In [82]:
comclidf = clidf[['Clinic_name', 'Address', 'City', 'ZipCode', 'fml']]

In [83]:
coltolower = ['Clinic_name', 'Address', 'City', 'ZipCode']

In [84]:
comclidf2 = pd.DataFrame()

In [85]:
for col in coltolower:
    comclidf2[col] = comclidf[col].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [86]:
comclidf2['fml'] = comclidf['fml']

In [87]:
comclidf2

,Clinic_name,Address,City,ZipCode,fml
0,elite biomechanical design,2208 5th ave,oroville,95965,"NICHOLAS,D.,MADSEN"
1,jennings orthopedic inc,10683 magnolia ave,riverside,92505,"HENRY,ALVIN,JENNINGS"
2,baldwin orthotics & prosthetics inc,24475 sunnymead blvd,moreno valley,92553,"HARLAN,L,BALDWIN"
3,"ferraco, inc.",2933 long beach blvd,long beach,90806,"PETER,L.,KORCHIN"
4,"pacific medical, inc.",3001 l st,sacramento,95816,"FERHAN,INAM,MAHER; DAVID,J.,SCURTI; GEORGE,,VI..."
...,...,...,...,...,...
58,"valley institute of prosthetics & orthotics, inc",23033 lyons ave,newhall,91321,"ALLEN,,DOLBERRY"
59,"valley institute of prosthetics & orthotics, inc.",1524 21st st,bakersfield,93301,"MICHAEL,KURT,NELSON; NICOLE,CAROLYN,KERR"
60,butte prosthetics,2260 esplanade,chico,95926,"RONALD,EDWARD,THURMAN"
61,active 1 prosthetics and orthotics,295 w cromwell ave,fresno,93711,"STEPHAN,M,MAGILL"


In [88]:
# comclidf2.to_excel('temp4.xlsx',index=False)

In [89]:
frame5.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,Phone_y,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11,NPI
0,"valley institute of prosthetics & orthotics, inc.",1524 21st st,bakersfield,ca,93301,6613221005,(661) 322-1005,NaN,NaN,http://www.vipoinc.com/,NaN,NaN,NaN,NaN,NaN,1912955881
1,channel islands prosthetics-orthotics,4517 market st,ventura,ca,93003,8056581822,(805) 658-1822,Fax (805) 658-1824,chris@cip-o.com,http://www.channelislandsp-o.com/,"John Stephen McAtee, CP, FAAOP,",Owner,"Arnel Dobrin-McAtee, CP Christopher J. Hollowa...",NaN,NaN,1891902524


In [90]:
frame6 = frame5.merge(comclidf2, on = ['Clinic_name', 'City'], how = 'left')


In [91]:
# frame6

In [92]:
frame6.columns

Index(['Clinic_name', 'Address_x', 'City', 'State', 'ZipCode_x', 'Phone',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'NPI', 'Address_y',
       'ZipCode_y', 'fml'],
      dtype='object')

In [93]:
frame6.rename(columns={'Address_x': 'Address', 
                            'ZipCode_x': 'ZipCode'}, inplace=True)

In [94]:
frame7 = frame6[['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'fml']]

In [95]:
frame7.to_excel('output_calist.xlsx',index=False)

In [96]:
frame7.shape

(401, 16)

# Load result_df  - use zip code to filter clinics by page

In [135]:
result_df = pd.read_excel('output_calist.xlsx')

In [136]:
# result_df['ZipCode'] = pd.to_numeric(result_df['ZipCode'], errors='coerce', downcast='integer')

In [137]:
area1 = [91934,92259,91905,92273,92231,92232,92243,91906,91987,92244,92249,91963,91962,91980,91948,92251,92036,91917,92250,91931,91901,91935,91916,92281,92274,91903,92227,92004,92019,91915,92154,91978,91914,91913,92021,92173,92040,91902,91977,91908,91911,92022,91976,91979,92020,91910,92233,91941,92275,91921,92153,92139,91946,92072,91945,92114,91943,91944,91909,91912,91942,91932,91933,91950,92119,91951,92065,92115,92257,92105,92071,92182,92136,92113,92120,92102,92066,92104,92155,92124,92118,92070,92134,92116,85349,92137,92138,92142,92143,92149,92150,92158,92159,92160,92161,92163,92165,92166,92167,92168,92170,92171,92172,92174,92175,92176,92177,92179,92186,92187,92191,92192,92193,92195,92196,92197,92112,92132,92147,85336,92178,92283,92108,92101,92123,92103,92086,92074,92064,92131,92135,92145,92140,92111,92110,92106,85350,92152,92199,92128,92117,92107,92126,92025,92198,92109,92129,92122,92027,92169,92121,92127,92093,85364,92037,92254,92130,92092,92060,92038,92039,92030,92033,92046,92014,92029,92067,92091,85369,92082,92061,92075,92096,92078,92007,92079,92024,92026,92069,92009,92023,85365,92222,92013,92059,85366,92081,92011,92084,92253,92539,92010,92536,92085,92083,92247,92248,92236,92008,92201,92561,92056,92210,92003,92202,92260,92018,85367,92057,92255,92261,92203,92054,92049,92051,92052,92211,92264]

In [138]:
area2 = [92879,92877,92878,92860,92505,92881,92880,92503,92882,91752,92504,91708,92883,92502,92513,92514,92516,92517,92519,92522,92509,92676,91761,92862,92506,92887,91710,92501,91762,92808,91709,92508,92337,92521,91743,92331,92507,91764,92570,91758,92610,92335,92518,92886,92316,92807,91730,92602,92885,92678,92334,92869,92823,91729,91763,91785,92313,92553,92861,91739,92557,91766,91786,92811,92867,91769,92376,92679,92532,92551,92620,92618,91765,91767,92870,92324,92782,92688,92630,91701,92865,92871,92336,92531,92705,91737,92572,92599,92866,92781,92650,92780,92357,91768,92350,92856,92857,92859,92863,92864,92806,91711,92530,92571,91784,92604,92354,92403,92411,92692,92410,92821,92587,91789,92401,92735,92318,92606,92831,92822,92691,92701,92408,92377,92834,92836,92837,92838,92868,92690,92402,92406,92413,92415,92418,92423,92427,91788,92799,92702,92711,92712,92835,92706,92675,92614,92805,92803,92812,92814,92815,92816,92817,92825,92850,92405,92899,91750,92637,92616,92619,92623,92698,91748,92609,92653,92585,92832,92612,92802,92586,91773,91724,91792,92707,92603,92373,92694,92703,92840,92375,91791,90632,90633,92842,92846,92595,92697,91740,92656,92809,92843,92617,92374,92555,92801,90631,92704,91723,92833,92369,92404,92556,91744,92584,92567,91759,92552,92626,91747,91749,91741,92554,92660,91722,92548,92804,92677,92657,90621,92346,91793,92358,92708,92841,90622,90624,92844,91790,90603,92728,91745,92322,92651,90638,92693,90680,92625,92627,90620,90604,90637,92607,92652,92654,92628,92655,90639,92685,91702,92562,90605,92683,92407,92662,92684,92320,92325,91746,92661,91706,92663,92646,92545,90602,92658,92659,92845,92605,92564,92673,90630,92647,90607,90608,90609,90623,90601,92629,92378,92582,91010,90670,92391,91732,92563,92624,92223,90703,92359,92615,90720,91009,91734,91735,90650,90716,91008,90606,92317,90610,90721,90701,91016,90651,90652,90715,92648,90702,92596,92382,91733,92399,92674,92649,90671,92397,91017,91731,92590,90740,90660,92385,92321,90742,90661,90662,92352,90713,90808,91780,90707,91006,90743,92546,92581,91771,91772,91066,91077,91770,92672,92543,90815,90240,91007,90640,90840,90706,90241,90711,90714,90239,92589,92593,90242,92591,91776,91755,90712,90803,92583,91025,91024,92344,90804,90814,91775,91756,91778,90202,93563,92341,92055,90723,90040,90831,92028,90091,90833,91716,90755,90022,90201,91714,91715,90805,91802,91896,91899,90807,92345,91754,91804,90806,91108,91107,91801,91803,92329,90280,92371,90270,92592,90221,91106,90801,90809,90832,90842,90844,90846,90847,90848,90853,90262,92088,91023,91031,91199,91030,90063,91104,91101,90813,92372,90023,90223,90224,90032,90895,91102,91109,91110,91114,91115,91116,91117,91118,91121,91124,91125,91126,91129,91182,91184,91185,91188,91189,90255,90058,92340,90810,92220,91123,90802,91011,91003,90222,92544,90033,90220,91105,91001,92068,90059,90042,90096,90002,91103,90001,90031,90746,90747,92392,90021,90745,92305,90013,90079,90041,92339,90744,90009,90030,90050,90051,90052,90053,90054,90055,90060,90072,90074,90075,90076,90078,90080,90081,90082,90083,90086,90087,90088,90093,90748,90012,90011,92058,90061,90014,90822,90189,93544,90099,90003,90065,90071,90749,92230,92315,90248,92308,90015,92333,90017,90502,90731,90044,90026,91206,90247,90070,90007,90037,92549,91205,90710,90089,90057,91012,90039,93553,92314,90733,90734,90047,90501,90006,90249,91209,91221,91222,91224,91225,91226,91204,91210,90062,92003,91021,90717,90504,90029,92057,90732,92395,90506,90303,91203,91020,90305,90018,90005,91208,90010,91046,90020,90507,90508,90509,90510,91202,90027,90004,91207,90250,90043,90251,91393,90503,92393,90260,90311,92386,90306,90307,90308,90309,90310,90312,90505,91214,90008,90302,92059,90019,90304,90038,90301,91201,90028,90278,93543,90275,92049,92051,92052,90016,90261,92054,90036,91042,92536,90274,91334,90056,91502,91503,91507,91508,91510,91526,90068,92394,91521,90277,92056,91501,91506,91523,91522,90254,90266,90267]

In [139]:
area3 = [92252,92286,92284,92277,92278,92241,92268,92256,92276,92240,92203,92211,92285,92258,92234,92202,92270,92201,92236,92235,92263,92262,92282,92255,92261,92210,92247,92248,92253,92260,92264,92304,92561,92386,92339,92230,92305,92220,92314,92549,92315,92254,92333,92239,92356,92399,92338,92583,92223,92539,92359,92320,92581,92341,92546,92582,92544,92554,92385,92382,92556,92543,92365,92555,92321,92373,92346,92536,92374,92375,92308,92332,92552,92567,92352,92275,92545,92317,92548,92369,92257,92327,92391,92378,92318,92307,92354,92408,92357,92596,92350,92571,92324,92551,92404,92557,92553,92592,92325,92401,92410,92585,92402,92406,92413,92415,92418,92423,92427,92066,92405,92407,92403,92572,92599,92313,92322,92518,92411,92586,92086,92340,92507,92345,92584,92521,92274,92563,92508,92591,92393,92395,92004,92060,92506,92312,92587,92376,92377,92501,92316,92398,92311,92394,92059,92570,92233,92502,92513,92514,92516,92517,92519,92522,92344,92368,92589,92593,92061,92532,92334,92595,92564,92504,92331,92392,92336,92509,92335,92531,92070,92337,92309,92503,92280,92562,92590,92082,92281,92505,91739,92881,92342,92358,91752,91759,92530,92323,91730,91737,92883,92879,92371,92397,91701,91764,92860,91729,91743,91761,92088,92329,92877,92878,92301,92028,92026,92310,92027,92880,92003,91762,92065,91785,92372,91758,91784,92227,92036,91786,92882,91708,91710,92676,92679,92068,92025,92069,92364,91763,92084,92678,91711,92030,92033,92046,92363,92675,92057,91916,91767,92055,92079,92347,92085,92083,92688,92251,91709,92096,91750,91931,92058,91769,91948,92862,92610,92887,91766,92078,92056,92029,92081,92808,92225,91962,92692,91768,93544,92366,92064,92226,92198,92630,92010,91773,92691,92672,92694,92690,92886,92040,92602,92823,91765,92127,92128,92807,92054,92008,92885,92283,92009,92618,91741,92074,92273,92869,92049,92051,92052,92673,91740,92199,92259,92013,92266,92620,91724,92609,92018,91903,92653,92693,92674,92011,91789,91905,92861,92650,93524,92811,92637,85371,91788,92867,92067]

In [140]:
zips_ca = pd.read_csv("zip-code-list.csv")

In [141]:
92074 in zips_ca["zip_code"].tolist()

True

In [142]:
len(area1) + len(area2) + len(area3)

1190

In [143]:
len(zips_ca["zip_code"])

2664

In [144]:
leftset = set(zips_ca["zip_code"]) - set(area1) - set(area2) - set(area3)
area4 = list(leftset)

In [146]:
# result_df

In [147]:
filtered1 = result_df[result_df['ZipCode'].isin(area1)]
filtered1.shape
# filtered1

(32, 16)

In [148]:
filtered2 = result_df[result_df['ZipCode'].isin(area2)]
filtered2.shape
# filtered2

(161, 16)

In [149]:
filtered3 = result_df[result_df['ZipCode'].isin(area3)]
filtered3.shape
# filtered3

(70, 16)

In [150]:
filtered4 = result_df[result_df['ZipCode'].isin(area4)]
filtered4.shape
# filtered4

(203, 16)

In [151]:
workbook = openpyxl.load_workbook('cleancali3.xlsx')

In [152]:
page_names = ['area1', 'area2', 'area3', 'area4']
dataframes = [filtered1, filtered2, filtered3, filtered4]

In [153]:
for df, page_name in zip(dataframes, page_names):
    new_worksheet = workbook.create_sheet(title=page_name)

    # Write DataFrame to the new worksheet
    for row in dataframe_to_rows(df, index=False, header=True):
        new_worksheet.append(row)

In [155]:
workbook.save('output_cabyzip.xlsx')